In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
import re


In [15]:
big_df = pd.read_csv('report_DP', sep=r' \| ', engine='python')
big_df

,algo_name,arch,scale,q,reln,limit,reps,eps,pre_eps,run_eps,nosy_ans,gsens,bias_ans,true_ans,nosy_ans_list,bias_ans_list,gsens_list
0,TSensDP,tpch,0.01,q1,CUSTOMER,100,10,1.0,0.5,0.5,5.793534e+04,122.4,57890.9,60175.0,"[56060.09275246905, 60670.625511070124, 58611....","[56108.0, 60175.0, 58888.0, 56108.0, 56108.0, ...","[108, 152, 122, 108, 109, 130, 103, 134, 116, ..."
1,TSensDP,tpch,0.01,q2,SUPPLIER,500,10,1.0,0.5,0.5,4.537502e+04,624.5,45151.5,60175.0,"[43209.79138345593, 59707.58210630711, 10647.0...","[43695.0, 59507.0, 8992.0, 48631.0, 56882.0, 4...","[614, 665, 575, 621, 649, 621, 646, 600, 620, ..."
2,TSensDP,tpch,0.01,q3,CUSTOMER,10,10,1.0,0.5,0.5,2.267290e+03,15.3,2266.5,2333.0,"[2366.0684026360095, 2135.6070261915247, 2232....","[2262.0, 2122.0, 2212.0, 2333.0, 2262.0, 2262....","[10, 8, 9, 23, 10, 10, 10, 25, 37, 11]"
3,PrivateSQL,tpch,0.01,q1,CUSTOMER,None,10,1.0,0.5,0.5,5.900351e+04,245.4,59024.5,60175.0,"[60505.546588959114, 58249.738738067266, 59748...","[60175, 58240, 59560, 59560, 60175, 60175, 575...","[252, 252, 465, 217, 224, 238, 189, 232, 189, ..."
4,PrivateSQL,tpch,0.01,q2,SUPPLIER,None,10,1.0,0.5,0.5,-4.699177e+02,725.7,0.0,60175.0,"[-1800.3581706712469, -222.12872440775323, 69....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[567, 456, 960, 810, 810, 640, 492, 830, 972, ..."
5,PrivateSQL,tpch,0.01,q3,CUSTOMER,None,10,1.0,0.5,0.5,-6.205124e+06,12035000.0,2301.3,2333.0,"[-25467069.110276617, 33968075.253394954, -104...","[2259, 2333, 2333, 2333, 2311, 2311, 2306, 228...","[11200000, 17100000, 11900000, 13300000, 10850..."


In [42]:
def num_symbol(num):
    return '+' if num > 0 else '' 

def beautify_float(f):
    return '%3.0f'%f

bf = beautify_float        

In [50]:
q_descp = {
    'q1': 'path join',
    'q2': 'acyclic join',
    'q3': 'cyclic join'
}

print('query', 'type', 'true answer', 'bias', 'sensitivity', 'error', 'bias', 'sensitivity', 'error', sep='\t')

for q in ['q1', 'q2', 'q3']:
    df_TSensDP = big_df.query('algo_name == "{algo_name}" & q == "{q}"'.format(algo_name='TSensDP', q=q))
    df_PrivateSQL = big_df.query('algo_name == "{algo_name}" & q == "{q}"'.format(algo_name='PrivateSQL', q=q))
    df_list = [df_TSensDP, df_PrivateSQL]
    
    cols = []
    for df in df_list:
        #print(q, df['algo_name'].tolist()[0])
        true_ans = int(df['true_ans'].tolist()[0])
        nosy_ans = df['nosy_ans'].tolist()[0]
        gsens = df['gsens'].tolist()[0]
        reln = df['reln'].tolist()[0]
        
        bias_ans_list = np.array(eval(str(df['bias_ans_list'].tolist()[0])))
        bias_ans_list = np.abs(bias_ans_list - true_ans) / true_ans
        bias = np.mean(bias_ans_list)
        bias = '%.1f%%'%(bias*100)
        #print(bias)
        
        nosy_ans_list = np.array(eval(str(df['nosy_ans_list'].tolist()[0])))
        err_list = np.abs(nosy_ans_list - true_ans) / true_ans
        error = np.mean(err_list)
        error = '%.1f%%'%(error*100)
        #print(error)
        
        cols += [bias, bf(gsens), error]
    cols = [q, q_descp[q], true_ans] + cols
    print(*cols, sep='\t')

query	desciprtion	true answer	bias	sensitivity	error	bias	sensitivity	error
q1	path join	60175	3.8%	122	4.0%	1.9%	245	2.2%
q2	acyclic join	60175	25.0%	624	24.6%	100.0%	726	100.8%
q3	cyclic join	2333	2.9%	 15	3.2%	1.4%	12035000	1157326.6%
